### Import and global settings

In [ ]:
import seaborn as sn
import numpy as np
import pandas as pd
import json

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from collections import Counter

### Pull data

### Process data

### Model choose and fit

### Print results